In [3]:
!pip install catboost lightgbm

In [85]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
test = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")
df_new = df.drop("target", axis = 1)

In [86]:
df

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,target,Id
0,8,14,4,1334.0,1440.0,XE,430,N13553,54.0,FAT,SAN,314,3.0,9.0,0,FYNC08ZRT1MS
1,5,3,6,1706.0,2016.0,XE,2567,N12924,103.0,BNA,EWR,748,9.0,18.0,1,4M3EPJ0BFKNV
2,7,11,5,1323.0,1507.0,XE,2989,N19966,125.0,CLT,IAH,913,6.0,33.0,1,OR30D5LJS4AF
3,12,11,4,2039.0,2238.0,WN,3743,N215WN,224.0,BWI,DEN,1491,5.0,10.0,1,0PRTFL3QD5ZI
4,5,23,5,1628.0,1744.0,UA,926,N347UA,60.0,LAX,SFO,337,3.0,13.0,1,DR7UZO31NC2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,1119.0,1238.0,AA,2421,N383AA,174.0,DFW,LAX,1235,10.0,15.0,0,KHV6OJ08LE0S
1999996,2,25,1,759.0,858.0,AA,1109,N431AA,36.0,DFW,AUS,190,6.0,17.0,0,WM56NFH3B82U
1999997,9,23,2,1226.0,2109.0,DL,162,N3763D,293.0,SEA,JFK,2421,30.0,20.0,0,K6UIWFZ0R1T7
1999998,7,26,6,729.0,929.0,US,1099,N956UW,97.0,PHL,JAX,742,3.0,20.0,0,RMFGZOY59UCK


In [87]:
columns = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

In [88]:
CatBoost = CatBoostClassifier(cat_features = columns, depth = 12, iterations = 970)

In [89]:
target = df["target"]

In [90]:
df_new = df_new.fillna(0)

In [91]:
df_new['DepTime'] = df_new['DepTime'] / 100

In [92]:
df_new['ArrTime'] = df_new['ArrTime'] / 100

In [93]:
origin_mte = df[['Origin', 'target']].groupby('Origin')['target'].mean()
origin_mte.name = 'Origin_MTE'
df_new = pd.merge(df_new, origin_mte, how='left', on='Origin')

In [94]:
Dest_mte = df[['Dest', 'target']].groupby('Dest')['target'].mean()
Dest_mte.name = 'Dest_MTE'
df_new = pd.merge(df_new, Dest_mte, how='left', on='Dest')

In [95]:
df_new = df_new.drop("Id", axis = 1)
df_new = df_new.drop("TailNum", axis = 1)

In [96]:
smth = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']
df.insert(loc=len(df.columns), column='Smth', value=smth) 

In [97]:
df_new.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,14,4,13.34,14.40,XE,430,54.0,FAT,SAN,314,3.0,9.0,0.203920,0.301685
1,5,3,6,17.06,20.16,XE,2567,103.0,BNA,EWR,748,9.0,18.0,0.302108,0.418648
2,7,11,5,13.23,15.07,XE,2989,125.0,CLT,IAH,913,6.0,33.0,0.297510,0.250599
3,12,11,4,20.39,22.38,WN,3743,224.0,BWI,DEN,1491,5.0,10.0,0.314572,0.277956
4,5,23,5,16.28,17.44,UA,926,60.0,LAX,SFO,337,3.0,13.0,0.289365,0.344834


In [98]:
Cat = CatBoost.fit(df_new, target)

Learning rate set to 0.272022
0:	learn: 0.6173761	total: 7.17s	remaining: 1h 55m 52s
1:	learn: 0.5680448	total: 11.7s	remaining: 1h 34m 37s
2:	learn: 0.5435802	total: 16.4s	remaining: 1h 28m 18s
3:	learn: 0.5298348	total: 21.4s	remaining: 1h 26m 19s
4:	learn: 0.5219857	total: 26.1s	remaining: 1h 23m 56s
5:	learn: 0.5143468	total: 30.4s	remaining: 1h 21m 19s
6:	learn: 0.5094081	total: 37.3s	remaining: 1h 25m 27s
7:	learn: 0.5063637	total: 41.6s	remaining: 1h 23m 16s
8:	learn: 0.5036080	total: 45.3s	remaining: 1h 20m 41s
9:	learn: 0.5010746	total: 49.8s	remaining: 1h 19m 41s
10:	learn: 0.4995314	total: 53.9s	remaining: 1h 18m 15s
11:	learn: 0.4975044	total: 57.9s	remaining: 1h 16m 58s
12:	learn: 0.4951057	total: 1m 2s	remaining: 1h 16m 46s
13:	learn: 0.4937537	total: 1m 6s	remaining: 1h 16m 3s
14:	learn: 0.4918777	total: 1m 11s	remaining: 1h 15m 22s
15:	learn: 0.4910500	total: 1m 15s	remaining: 1h 14m 50s
16:	learn: 0.4893244	total: 1m 19s	remaining: 1h 14m 14s
17:	learn: 0.4884800	total

145:	learn: 0.3858776	total: 11m 24s	remaining: 1h 4m 23s
146:	learn: 0.3856386	total: 11m 29s	remaining: 1h 4m 19s
147:	learn: 0.3852961	total: 11m 34s	remaining: 1h 4m 18s
148:	learn: 0.3844580	total: 11m 39s	remaining: 1h 4m 13s
149:	learn: 0.3842793	total: 11m 44s	remaining: 1h 4m 11s
150:	learn: 0.3837159	total: 11m 50s	remaining: 1h 4m 11s
151:	learn: 0.3833434	total: 11m 54s	remaining: 1h 4m 6s
152:	learn: 0.3831434	total: 11m 59s	remaining: 1h 4m 4s
153:	learn: 0.3827576	total: 12m 4s	remaining: 1h 3m 56s
154:	learn: 0.3826214	total: 12m 9s	remaining: 1h 3m 55s
155:	learn: 0.3817784	total: 12m 13s	remaining: 1h 3m 49s
156:	learn: 0.3810875	total: 12m 18s	remaining: 1h 3m 44s
157:	learn: 0.3808290	total: 12m 24s	remaining: 1h 3m 46s
158:	learn: 0.3802923	total: 12m 29s	remaining: 1h 3m 42s
159:	learn: 0.3797650	total: 12m 34s	remaining: 1h 3m 39s
160:	learn: 0.3788498	total: 12m 39s	remaining: 1h 3m 35s
161:	learn: 0.3784095	total: 12m 44s	remaining: 1h 3m 31s
162:	learn: 0.3780

291:	learn: 0.3301283	total: 23m 28s	remaining: 54m 29s
292:	learn: 0.3298789	total: 23m 32s	remaining: 54m 24s
293:	learn: 0.3297047	total: 23m 37s	remaining: 54m 19s
294:	learn: 0.3296042	total: 23m 41s	remaining: 54m 12s
295:	learn: 0.3294319	total: 23m 45s	remaining: 54m 6s
296:	learn: 0.3291394	total: 23m 50s	remaining: 54m 1s
297:	learn: 0.3286877	total: 23m 54s	remaining: 53m 55s
298:	learn: 0.3281769	total: 23m 59s	remaining: 53m 51s
299:	learn: 0.3279708	total: 24m 4s	remaining: 53m 45s
300:	learn: 0.3278524	total: 24m 8s	remaining: 53m 40s
301:	learn: 0.3277340	total: 24m 13s	remaining: 53m 34s
302:	learn: 0.3276017	total: 24m 17s	remaining: 53m 29s
303:	learn: 0.3275804	total: 24m 22s	remaining: 53m 24s
304:	learn: 0.3273215	total: 24m 27s	remaining: 53m 20s
305:	learn: 0.3265445	total: 24m 31s	remaining: 53m 13s
306:	learn: 0.3258678	total: 24m 36s	remaining: 53m 8s
307:	learn: 0.3251170	total: 24m 41s	remaining: 53m 4s
308:	learn: 0.3245663	total: 24m 46s	remaining: 52m 59

439:	learn: 0.2937073	total: 35m 12s	remaining: 42m 24s
440:	learn: 0.2935120	total: 35m 16s	remaining: 42m 18s
441:	learn: 0.2934487	total: 35m 22s	remaining: 42m 15s
442:	learn: 0.2930699	total: 35m 27s	remaining: 42m 10s
443:	learn: 0.2928340	total: 35m 32s	remaining: 42m 5s
444:	learn: 0.2925880	total: 35m 36s	remaining: 42m
445:	learn: 0.2921662	total: 35m 40s	remaining: 41m 55s
446:	learn: 0.2919304	total: 35m 45s	remaining: 41m 50s
447:	learn: 0.2917868	total: 35m 50s	remaining: 41m 45s
448:	learn: 0.2916670	total: 35m 54s	remaining: 41m 40s
449:	learn: 0.2915015	total: 35m 59s	remaining: 41m 35s
450:	learn: 0.2912116	total: 36m 4s	remaining: 41m 31s
451:	learn: 0.2908781	total: 36m 10s	remaining: 41m 26s
452:	learn: 0.2908085	total: 36m 14s	remaining: 41m 21s
453:	learn: 0.2907229	total: 36m 19s	remaining: 41m 17s
454:	learn: 0.2905873	total: 36m 24s	remaining: 41m 12s
455:	learn: 0.2904355	total: 36m 29s	remaining: 41m 7s
456:	learn: 0.2903749	total: 36m 34s	remaining: 41m 3s


587:	learn: 0.2679655	total: 46m 50s	remaining: 30m 25s
588:	learn: 0.2678676	total: 46m 55s	remaining: 30m 21s
589:	learn: 0.2677820	total: 47m	remaining: 30m 16s
590:	learn: 0.2676866	total: 47m 4s	remaining: 30m 11s
591:	learn: 0.2674893	total: 47m 9s	remaining: 30m 6s
592:	learn: 0.2671457	total: 47m 13s	remaining: 30m 1s
593:	learn: 0.2670239	total: 47m 17s	remaining: 29m 56s
594:	learn: 0.2668820	total: 47m 23s	remaining: 29m 52s
595:	learn: 0.2666635	total: 47m 28s	remaining: 29m 47s
596:	learn: 0.2666123	total: 47m 33s	remaining: 29m 43s
597:	learn: 0.2664276	total: 47m 39s	remaining: 29m 38s
598:	learn: 0.2662752	total: 47m 44s	remaining: 29m 34s
599:	learn: 0.2661324	total: 47m 49s	remaining: 29m 29s
600:	learn: 0.2659370	total: 47m 53s	remaining: 29m 24s
601:	learn: 0.2657881	total: 47m 57s	remaining: 29m 19s
602:	learn: 0.2653366	total: 48m 2s	remaining: 29m 14s
603:	learn: 0.2650816	total: 48m 7s	remaining: 29m 9s
604:	learn: 0.2648029	total: 48m 12s	remaining: 29m 4s
605:

735:	learn: 0.2435002	total: 58m 50s	remaining: 18m 42s
736:	learn: 0.2433910	total: 58m 54s	remaining: 18m 37s
737:	learn: 0.2432494	total: 58m 58s	remaining: 18m 32s
738:	learn: 0.2431238	total: 59m 3s	remaining: 18m 27s
739:	learn: 0.2430354	total: 59m 8s	remaining: 18m 22s
740:	learn: 0.2429303	total: 59m 12s	remaining: 18m 17s
741:	learn: 0.2428130	total: 59m 18s	remaining: 18m 13s
742:	learn: 0.2424678	total: 59m 22s	remaining: 18m 8s
743:	learn: 0.2423842	total: 59m 27s	remaining: 18m 3s
744:	learn: 0.2421104	total: 59m 31s	remaining: 17m 58s
745:	learn: 0.2418456	total: 59m 36s	remaining: 17m 53s
746:	learn: 0.2417626	total: 59m 40s	remaining: 17m 48s
747:	learn: 0.2413470	total: 59m 45s	remaining: 17m 44s
748:	learn: 0.2412240	total: 59m 50s	remaining: 17m 39s
749:	learn: 0.2411411	total: 59m 56s	remaining: 17m 34s
750:	learn: 0.2409968	total: 1h 2s	remaining: 17m 30s
751:	learn: 0.2408549	total: 1h 7s	remaining: 17m 25s
752:	learn: 0.2406733	total: 1h 11s	remaining: 17m 20s
7

880:	learn: 0.2257694	total: 1h 10m 30s	remaining: 7m 7s
881:	learn: 0.2256444	total: 1h 10m 35s	remaining: 7m 2s
882:	learn: 0.2254857	total: 1h 10m 41s	remaining: 6m 57s
883:	learn: 0.2253950	total: 1h 10m 46s	remaining: 6m 53s
884:	learn: 0.2252927	total: 1h 10m 51s	remaining: 6m 48s
885:	learn: 0.2251893	total: 1h 10m 56s	remaining: 6m 43s
886:	learn: 0.2251484	total: 1h 11m 2s	remaining: 6m 38s
887:	learn: 0.2250562	total: 1h 11m 7s	remaining: 6m 34s
888:	learn: 0.2249796	total: 1h 11m 11s	remaining: 6m 29s
889:	learn: 0.2248962	total: 1h 11m 16s	remaining: 6m 24s
890:	learn: 0.2248422	total: 1h 11m 21s	remaining: 6m 19s
891:	learn: 0.2247631	total: 1h 11m 26s	remaining: 6m 14s
892:	learn: 0.2246714	total: 1h 11m 31s	remaining: 6m 10s
893:	learn: 0.2246311	total: 1h 11m 36s	remaining: 6m 5s
894:	learn: 0.2245862	total: 1h 11m 41s	remaining: 6m
895:	learn: 0.2244868	total: 1h 11m 45s	remaining: 5m 55s
896:	learn: 0.2242144	total: 1h 11m 50s	remaining: 5m 50s
897:	learn: 0.2240462	t

In [99]:
test = test.fillna(0)

In [100]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Id
0,8,15,5,806.0,957.0,UA,1157,N834UA,111.0,DFW,DEN,641,8.0,52.0,QKXNUJO50EPF
1,10,14,2,2020.0,2133.0,WN,1965,N203WN,49.0,PHX,ONT,325,4.0,20.0,Y2XFDOHQK6VM
2,6,1,7,2245.0,9.0,FL,910,N943AT,67.0,ATL,PHF,508,6.0,11.0,TU5QMACJHR10
3,12,31,3,1638.0,1753.0,OO,5732,N567SW,66.0,GEG,SEA,224,3.0,6.0,0731ET5Z9FWJ
4,10,23,4,1544.0,2358.0,AA,22,N329AA,289.0,LAX,JFK,2475,12.0,13.0,5BNGICLQFKSO


In [101]:
test = test.drop("TailNum", axis = 1)

In [102]:
test = test.fillna(0)

In [103]:
test['DepTime'] = test['DepTime'] / 100

In [104]:
test['ArrTime'] = test['ArrTime'] / 100

In [105]:
test = test.drop("Id", axis = 1)

In [106]:
df["Smth2"] = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']

In [107]:
test["Origin_MTE"] = df_new["Origin_MTE"]
test["Dest_MTE"] = df_new["Dest_MTE"]

In [108]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,15,5,8.06,9.57,UA,1157,111.0,DFW,DEN,641,8.0,52.0,0.203920,0.301685
1,10,14,2,20.20,21.33,WN,1965,49.0,PHX,ONT,325,4.0,20.0,0.302108,0.418648
2,6,1,7,22.45,0.09,FL,910,67.0,ATL,PHF,508,6.0,11.0,0.297510,0.250599
3,12,31,3,16.38,17.53,OO,5732,66.0,GEG,SEA,224,3.0,6.0,0.314572,0.277956
4,10,23,4,15.44,23.58,AA,22,289.0,LAX,JFK,2475,12.0,13.0,0.289365,0.344834


In [109]:
proba = CatBoost.predict_proba(test)[:, 1]

In [110]:
submission = pd.read_csv("sample_submission.csv")

In [111]:
submission['target'] = proba

In [112]:
submission.to_csv("submission.csv", index = False, header = True)